In [1]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarcnr
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser
from scipy.stats import entropy

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Wed Aug  3 00:44:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 68%   79C    P2   134W / 260W |   5939MiB / 11019MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 30%   

# MongoDB

In [4]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    db_cn = db.lda_topic_cn
    
    
except errors.ConnectionFailure as err:
    print(err)

In [5]:
db_cn

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'lda_topic_cn')

# CN

In [6]:
cn_topic = db_cn.aggregate([
        {"$unwind" : "$classification"}
])


In [7]:
cn_df = pd.DataFrame(cn_topic)

In [8]:
# 取cpc前3碼
cn_df["cpc_n"] = cn_df.classification.apply(lambda x: x[0:3])

In [9]:
cn_df

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n
0,62c6c855efb892ae553bd9fc,CN102140507B,2010-12-20 05:00:00,[Nankai Univ],Y02A50/30,本发明提供了一种关于引起人类感染性腹泻的主要病原微生物的基因芯片及检测用试剂盒，其主要针对致...,1,Y02
1,62c6c855efb892ae553bd9ff,CN102080127A,2010-09-06 04:00:00,"[GUANGZHOU IMPROVE MEDICAL CO Ltd, Wuhan Unive...",Y02A50/30,本发明公开了一种检测15种临床常见病原微生物的基因芯片。该基因芯片包括固定在固相载体上的特异...,5,Y02
2,62c6c855efb892ae553bda01,CN103003854B,2010-07-29 04:00:00,[Ford Global Technologies LLC],G06F17/00,可检查车辆的动力操控状态、驾驶员到车辆的输入等来确定驾驶员工作负担的一个或多个测量。然后，可...,8,G06
3,62c6c855efb892ae553bda01,CN103003854B,2010-07-29 04:00:00,[Ford Global Technologies LLC],B60W50/087,可检查车辆的动力操控状态、驾驶员到车辆的输入等来确定驾驶员工作负担的一个或多个测量。然后，可...,8,B60
4,62c6c855efb892ae553bda01,CN103003854B,2010-07-29 04:00:00,[Ford Global Technologies LLC],G01C21/3641,可检查车辆的动力操控状态、驾驶员到车辆的输入等来确定驾驶员工作负担的一个或多个测量。然后，可...,8,G01
...,...,...,...,...,...,...,...,...
124985,62c6c855efb892ae553c67e5,CN111935889A,2020-09-30 04:00:00,[Guilin Xintong Technology Co ltd],Y02B20/40,本发明公开了一种路灯自动采集及远程控制装置，该装置包括控制模块和分别与控制模块相连接的采集模...,6,Y02
124986,62c6c855efb892ae553c67e7,CN111353035B,2020-03-11 04:00:00,[Mgjia Beijing Technology Co ltd],G06F16/3329,本公开涉及一种人机对话方法、装置、可读存储介质及电子设备。方法包括：在接收到用户当前输入的文...,3,G06
124987,62c6c855efb892ae553c67e7,CN111353035B,2020-03-11 04:00:00,[Mgjia Beijing Technology Co ltd],G06N3/0445,本公开涉及一种人机对话方法、装置、可读存储介质及电子设备。方法包括：在接收到用户当前输入的文...,3,G06
124988,62c6c855efb892ae553c67e7,CN111353035B,2020-03-11 04:00:00,[Mgjia Beijing Technology Co ltd],G06N3/0454,本公开涉及一种人机对话方法、装置、可读存储介质及电子设备。方法包括：在接收到用户当前输入的文...,3,G06


In [10]:
cn_df2 = cn_df.groupby(["cpc_n","topic"],as_index=False).size()

In [11]:
cn_df2

,cpc_n,topic,size
0,A01,0,11
1,A01,1,49
2,A01,2,5
3,A01,3,18
4,A01,4,14
...,...,...,...
795,Y10,4,2
796,Y10,5,54
797,Y10,6,16
798,Y10,7,6


## 找出前10的CPC計算各類別的topic比例

In [12]:
cn_cpc_ls = ["H04","G06","G01","H01","Y02","G08","A61","G05","H02","C08"]

In [13]:
cn_cpc_top10 = cn_df2[cn_df2.cpc_n.isin(cn_cpc_ls)]
cn_cpc_top10

,cpc_n,topic,size
65,A61,0,117
66,A61,1,298
67,A61,2,27
68,A61,3,60
69,A61,4,38
...,...,...,...
776,Y02,5,2176
777,Y02,6,636
778,Y02,7,255
779,Y02,8,241


In [14]:
cn_cpc_sum = cn_cpc_top10.groupby(["cpc_n"],as_index=False).sum()[["cpc_n","size"]]
cn_cpc_sum.rename(columns = {'size':'sum'}, inplace = True)
cn_cpc_sum

,cpc_n,sum
0,A61,2784
1,C08,1549
2,G01,4907
3,G05,2241
4,G06,16107
5,G08,3600
6,H01,4313
7,H02,1484
8,H04,63140
9,Y02,4218


In [15]:
sum(cn_cpc_sum["sum"])

104343

In [16]:
cn_cpc_top10_fin = pd.merge(cn_cpc_top10, cn_cpc_sum, on="cpc_n")
cn_cpc_top10_fin

,cpc_n,topic,size,sum
0,A61,0,117,2784
1,A61,1,298,2784
2,A61,2,27,2784
3,A61,3,60,2784
4,A61,4,38,2784
...,...,...,...,...
95,Y02,5,2176,4218
96,Y02,6,636,4218
97,Y02,7,255,4218
98,Y02,8,241,4218


In [17]:
# 計算topic比例
cn_cpc_top10_fin["topic_ratio"] = cn_cpc_top10_fin.apply(lambda x: x["size"]/x["sum"],axis=1)

In [18]:
cn_cpc_top10_fin

,cpc_n,topic,size,sum,topic_ratio
0,A61,0,117,2784,0.042026
1,A61,1,298,2784,0.107040
2,A61,2,27,2784,0.009698
3,A61,3,60,2784,0.021552
4,A61,4,38,2784,0.013649
...,...,...,...,...,...
95,Y02,5,2176,4218,0.515884
96,Y02,6,636,4218,0.150782
97,Y02,7,255,4218,0.060455
98,Y02,8,241,4218,0.057136


## 計算Entropy
計算各CPC底下topic的混亂程度

In [19]:
entropy(list(cn_cpc_top10_fin[cn_cpc_top10_fin.cpc_n=="H04"]["topic_ratio"]), base=2)

2.0618238955798414

In [20]:
cn_entropy_ls = []
for i in cn_cpc_ls:
    en_num = entropy(list(cn_cpc_top10_fin[cn_cpc_top10_fin.cpc_n==i]["topic_ratio"]), base=2)
    cn_entropy_ls.append(en_num)

In [21]:
cn_entropy_ls

[2.0618238955798414,
 2.207646571522458,
 2.24256233726986,
 2.9094063979791183,
 2.2631568910528364,
 2.0254432635183077,
 2.287837536061102,
 2.067296722863456,
 2.448005355635949,
 2.7055934797274497]

In [22]:
cn_entropy = pd.DataFrame((zip(cn_cpc_ls, cn_entropy_ls)), columns = ['cpc', 'entropy'])
cn_entropy

,cpc,entropy
0,H04,2.061824
1,G06,2.207647
2,G01,2.242562
3,H01,2.909406
4,Y02,2.263157
5,G08,2.025443
6,A61,2.287838
7,G05,2.067297
8,H02,2.448005
9,C08,2.705593
